In [1]:
import pandas as pd
import numpy as np
import ee
ee.Authenticate()

True

In [ ]:
# Initialize Earth Engine
ee.Initialize(project='ee-nguyendangkhoi9517')

# Read DataFrame from CSV
df = pd.read_csv('Data_csv/flood_data.csv')

# Setup parameters
square_size_m = 10000  # 10 km² (10000m x 10000m)
small_square_size_m = 250  # 250m x 250m for sub-squares
event_results = []

# Loop through the rows in the DataFrame
for index, row in df[1:2].iterrows():
    center_lon = row['dfo_centroid_x']
    center_lat = row['dfo_centroid_y']
    area_km2 = row['gfd_area']  # Area

    # Calculate the radius for the region of interest (in meters)
    radius = np.sqrt(area_km2) * 1000 / 2  # Radius from the center

    # Skip if radius is too small or zero
    if radius <= 0:
        print(f"Skipping event {index} due to zero radius.")
        continue

    # Process begin and end dates
    try:
        began_date_raw = row['dfo_began']
        ended_date_raw = row['dfo_ended']

        # Convert 2-digit year if necessary
        if len(began_date_raw.split('/')[-1]) == 2:
            began_date_raw = began_date_raw[:-2] + '20' + began_date_raw[-2:]
        if len(ended_date_raw.split('/')[-1]) == 2:
            ended_date_raw = ended_date_raw[:-2] + '20' + ended_date_raw[-2:]

        began_date = pd.to_datetime(began_date_raw, format='%m/%d/%Y', errors='coerce')
        ended_date = pd.to_datetime(ended_date_raw, format='%m/%d/%Y', errors='coerce')

        if pd.isna(began_date) or pd.isna(ended_date):
            print(f"Invalid date format for event {index}. Skipping this event.")
            continue

        began_date_str = began_date.strftime('%Y-%m-%d')
        ended_date_str = ended_date.strftime('%Y-%m-%d')
    except Exception as e:
        print(f"Error parsing dates for event {index}: {e}")
        continue

    # Create the center point and a circular region around it
    point = ee.Geometry.Point(center_lon, center_lat)
    circle = point.buffer(radius)

    # Calculate the number of squares to cover the circular area
    num_squares = int(np.ceil(radius / square_size_m))

    # Create squares around the center point
    squares = []
    for i in range(-num_squares, num_squares + 1):
        for j in range(-num_squares, num_squares + 1):
            # Calculate the boundary for each 10 km² square
            square = ee.Geometry.Rectangle([
                center_lon + i * square_size_m / 111320 - square_size_m / 111320 / 2,
                center_lat + j * square_size_m / 111320 - square_size_m / 111320 / 2,
                center_lon + i * square_size_m / 111320 + square_size_m / 111320 / 2,
                center_lat + j * square_size_m / 111320 + square_size_m / 111320 / 2
            ])
            # Add square to the list if it intersects with the circle
            if circle.intersects(square):
                squares.append(square)

    print(f"Number of squares for event {index}: {len(squares)}")

    # Load DEM data (Copernicus GLO-30 DEM)
    dem = ee.ImageCollection('COPERNICUS/DEM/GLO30').select('DEM').mosaic()

    # Load CHIRPS Daily Precipitation dataset
    chirps = ee.ImageCollection('UCSB-CHG/CHIRPS/DAILY').select('precipitation')
    # Load flood event data (MODIS flood duration)
    flood_data = ee.ImageCollection('GLOBAL_FLOOD_DB/MODIS_EVENTS/V1').filterBounds(circle).filterDate(began_date, ended_date)

    # Store results for the current event
    for square_index, square in enumerate(squares):

        # Get rainfall mean for 3 days, 7 days, and 1 month
        rain_3_days = chirps.filterDate(began_date_str, (began_date + pd.Timedelta(days=3)).strftime('%Y-%m-%d'))
        rain_7_days = chirps.filterDate(began_date_str, (began_date + pd.Timedelta(days=7)).strftime('%Y-%m-%d'))
        rain_1_month = chirps.filterDate(began_date_str, (began_date + pd.Timedelta(days=30)).strftime('%Y-%m-%d'))

        try:
            rainfall_3d = rain_3_days.mean().reduceRegion(
                reducer=ee.Reducer.mean(),
                geometry=square,
                scale=5000,
                maxPixels=1e13
            ).getInfo().get('precipitation')

            rainfall_7d = rain_7_days.mean().reduceRegion(
                reducer=ee.Reducer.mean(),
                geometry=square,
                scale=5000,
                maxPixels=1e13
            ).getInfo().get('precipitation')

            rainfall_1m = rain_1_month.mean().reduceRegion(
                reducer=ee.Reducer.mean(),
                geometry=square,
                scale=5000,
                maxPixels=1e13
            ).getInfo().get('precipitation')
        except Exception as e:
            print(f"Error calculating rainfall for event {index}, square {square_index}: {e}")
            rainfall_3d, rainfall_7d, rainfall_1m = None, None, None

        # Calculate the center coordinates for the square
        square_center = square.centroid().coordinates().getInfo()
        square_center_lat = square_center[1]
        square_center_lon = square_center[0]

        # Get the bounds of the square (left, bottom, right, top)
        bounds = square.bounds().getInfo()['coordinates'][0]
        left, bottom, right, top = bounds[0][0], bounds[0][1], bounds[2][0], bounds[2][1]

        # Calculate the number of small squares (250m x 250m) within the 10 km² square
        num_small_squares_x = int((right - left) * 111320 / small_square_size_m)  # Number of small squares along x (longitude)
        num_small_squares_y = int((top - bottom) * 111320 / small_square_size_m)  # Number of small squares along y (latitude)

        small_squares = []
        for x in range(num_small_squares_x):
            for y in range(num_small_squares_y):
                small_square_left = left + x * small_square_size_m / 111320
                small_square_bottom = bottom + y * small_square_size_m / 111320
                small_square_right = small_square_left + small_square_size_m / 111320
                small_square_top = small_square_bottom + small_square_size_m / 111320
                small_square = ee.Geometry.Rectangle([small_square_left, small_square_bottom, small_square_right, small_square_top])
                small_squares.append(small_square)

        # Ensure there are exactly 160 small squares
        if len(small_squares) != 1600:
            print(f"Warning: Number of small squares is {len(small_squares)} instead of 1600.")

        # Get mean elevation for each small square
        height_values = []  # Reset height values for each square
        dem_square = dem.clip(square)
        for small_square in small_squares:
            # Calculate mean elevation for the 250m x 250m square
            mean_elevation = dem_square.reduceRegion(
                reducer=ee.Reducer.mean(),
                geometry=small_square,
                scale=small_square_size_m,  # 250m resolution
                maxPixels=1e13
            ).getInfo().get('DEM')

            # If elevation is available, add to the list
            if mean_elevation is not None:
                height_values.append(round(mean_elevation, 2))  # Round to 2 decimal places
            # Ensure there are exactly 1600 small squares
            if len(height_values) < 1600:
                mean_height = round(np.mean(height_values), 2) if height_values else 0  # Tính mean hoặc gán 0 nếu danh sách trống
                height_values.extend([mean_height] * (1600 - len(height_values)))

        # Ensure there are exactly 1600 small squares
        flood_duration_values = [0] * 1600  # Initialize with 1600 zeros
        flood_square = flood_data.filterBounds(square).mosaic()
        for idx, small_square in enumerate(small_squares[:1600]):  # Limit to 1600 squares
            # Calculate mean flood duration for the 250m x 250m square
            flood_duration = flood_square.select('duration').reduceRegion(
                reducer=ee.Reducer.mean(),
                geometry=small_square,
                scale=small_square_size_m,  # 250m resolution
                maxPixels=1e13
            ).getInfo()

            # If 'duration' exists, update the list
            if 'duration' in flood_duration and flood_duration['duration'] is not None:
                flood_duration_values[idx] = round(flood_duration['duration'], 2)  # Round to 2 decimal places

        print(f"Event Index: {index}, Square Index: {square_index}")
        print(f"Square Center Latitude: {square_center_lat}, Square Center Longitude: {square_center_lon}")
        print(f"Length of Height Values: {len(height_values)}")
        print(f"Length of Flood Duration values: {len(flood_duration_values)}")
        print(f"Rainfall 3d: {rainfall_3d}, 7d: {rainfall_7d}, 1m: {rainfall_1m}")
        print('-' * 50)

        # Store the results for the event
        event_results.append({
            'event_index': index,  # Add event_index
            'began_date': began_date_str,
            'ended_date': ended_date_str,
            'square_index': square_index,  # Add square_index
            'square_center_lat': square_center_lat,
            'square_center_lon': square_center_lon,
            'height_values': height_values, # Store the height values list
            'flood_duration_values': flood_duration_values,  # Store the flood duration values list
            'rainfall_3d': round(rainfall_3d, 2) if rainfall_3d is not None else None,
            'rainfall_7d': round(rainfall_7d, 2) if rainfall_7d is not None else None,
            'rainfall_1m': round(rainfall_1m, 2) if rainfall_1m is not None else None
        })

# Convert event results to DataFrame
event_results_df = pd.DataFrame(event_results)

# Merge the event results with the original DataFrame (optional, based on your needs)
# df = df.join(event_results_df.set_index(['event_index', 'square_index'])['height_values'], on=['dfo_centroid_y', 'dfo_centroid_x'])

# Save results in the DataFrame to CSV
event_results_output_path = 'Data_model/data_flood_sum.csv'
event_results_df.to_csv(event_results_output_path, index=False)
print(f"Results saved to: {event_results_output_path}")
